In [1]:
import pandas as pd
import numpy as np
from scipy.stats import t

In [2]:
def family_of_loss_func(actual, predicted, degree):
    """
    Implemented from:
    Patton, A. J., 2011. Volatility forecasting comparison using imperfect 
    volatility proxies, Journal of Econometrics 160, 246-256.
    """
    if degree == -2:
        # QLIKE
        loss = actual / predicted - np.log(actual / predicted) - 1
    elif degree == -1:
        loss = predicted - actual + actual * np.log(actual / predicted)
    else:
        # MSE if degree = 0
        loss = (np.sqrt(actual) ** (2 * degree + 4) - predicted ** (degree + 2)) / ((degree + 1) * (degree + 2))
        loss -= (1 / (degree + 1)) * (predicted ** (degree + 1)) * (actual - predicted)
    return loss

def panel_DM(act, pred1, pred2, degree = 0):
    """
    Implemented from:
    Timmermann, A., Zhu, Y., 2019. Comparing Forecasting Performance with Panel Data
    """
    l1 = family_of_loss_func(act, pred1, degree)
    l2 = family_of_loss_func(act, pred2, degree)
    d12 = l1 - l2
    d12n = np.nansum(d12, axis = 1)
    n = np.sum(~np.isnan(d12), axis = 1)
    T = d12.shape[0]
    nT = np.sum(~np.isnan(d12))
    hat_d12 = np.nansum(d12, axis = 1) / np.sum(~np.isnan(d12), axis = 1)
    R12 = np.sqrt(n) * hat_d12
    hat_R0 = np.nansum(R12) / T
    hat_R1 = np.nansum(R12[1:]) / (T - 1)
    hat_R11 = np.nansum(R12[:-1]) / (T - 1)
    g0 = np.nansum((R12 - hat_R0) * (R12 - hat_R0)) / T
    g1 = 2 * np.nansum((R12[1:] - hat_R11) * (R12[:-1] - hat_R1)) / (T - 1)
    sig = np.sqrt(g0 + g1)
    DM = np.nansum(d12) / (np.sqrt(nT) * sig)
    p_value = 2 * t.cdf(-np.abs(DM), df = nT - 1)
    return DM, p_value

df_act = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/daily_vola.csv')
df_act.set_index(df_act.iloc[:, 0], inplace = True)
df_act = df_act.iloc[:-1, 1:]
df_act.pop('AMCR')
df_pred1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/pgarch_csa/forecasts_all.csv')
df_pred1.set_index(df_pred1.iloc[:, 0], inplace = True)
df_pred1 = df_pred1.iloc[:, 1:-1].T
df_pred2 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/Panel_EWMA/forecast.csv')
df_pred2.set_index(df_pred2.iloc[:, 0], inplace = True)
df_pred2 = df_pred2.iloc[:, 1:].T
df_pred2 = df_pred2.replace(1., np.nan)
df_pred3 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/Panel_GARCH/forecast_norm_dist.csv')
df_pred3.set_index(df_pred3.iloc[:, 0], inplace = True)
df_pred3 = df_pred3.iloc[:, 1:].T
df_pred3.pop('AMCR')
df_pred4 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/Panel_GARCH/forecast_t_dist.csv')
df_pred4.set_index(df_pred4.iloc[:, 0], inplace = True)
df_pred4 = df_pred4.iloc[:, 1:].T
df_pred4.pop('AMCR')
df_pred5 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/Panel_GARCH_MIDAS/Concated/forecast_norm.csv')
df_pred5.set_index(df_pred5.iloc[:, 0], inplace = True)
df_pred5 = df_pred5.iloc[:, 1:]
df_pred6 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/Panel_GARCH_MIDAS/Concated/forecast_stud.csv')
df_pred6.set_index(df_pred6.iloc[:, 0], inplace = True)
df_pred6 = df_pred6.iloc[:, 1:]

## Diebold-Mariano Test for Panel Forecast:

In [3]:
panel_DM(df_act.values, df_pred6.values, df_pred1.values)

(-1.1532302139456434, 0.24881902452300098)

# Daily forecasts with the second edition of DM test:

### Panel GARCH with cross sectional adjustment vs EWMA:

In [4]:
panel_DM(df_act.values, df_pred1.values, df_pred2.values)

(1.4516914866703243, 0.14659095761319438)

### Panel GARCH (normal)  vs EWMA:

In [5]:
panel_DM(df_act.values, df_pred3.values, df_pred2.values)

(-1.0211712612485495, 0.30717610120501965)

### Panel GARCH (student-t) vs EWMA:

In [6]:
panel_DM(df_act.values, df_pred4.values, df_pred2.values)

(-1.0407454943319665, 0.2979965301392632)

### Panel GARCH-MIDAS (normal) vs EWMA:

In [7]:
panel_DM(df_act.values, df_pred5.values, df_pred2.values)

(0.6040634601965328, 0.5458030024264691)

### Panel GARCH-MIDAS (student-t) vs EWMA:

In [8]:
panel_DM(df_act.values, df_pred6.values, df_pred2.values)

(-0.0865649962718948, 0.9310174864781338)

### Panel GARCH (normal) vs GARCH with cross sectional adjustment

In [9]:
panel_DM(df_act.values, df_pred3.values, df_pred1.values)

(-1.3947126474480827, 0.16310601528238494)

### Panel GARCH (studnet-t) vs GARCH with cross sectional adjustment

In [10]:
panel_DM(df_act.values, df_pred4.values, df_pred1.values)

(-1.38969608572591, 0.16462468497178748)

### Panel GARCH-MIDAS (normal) vs GARCH with cross sectional adjustment

In [11]:
panel_DM(df_act.values, df_pred5.values, df_pred1.values)

(-0.9287364745014535, 0.3530282099970743)

### Panel GARCH-MIDAS (student-t) vs GARCH with cross sectional adjustment

In [12]:
panel_DM(df_act.values, df_pred6.values, df_pred1.values)

(-1.1532302139456434, 0.24881902452300098)

# Results from the first version of DM test:

##### Note: Which is In [2]

In [13]:
print('Daily MSE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/daily_mse_v3.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Daily MSE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(1.452, 0.147)","(1.395, 0.163)","(1.39, 0.165)","(0.929, 0.353)","(1.153, 0.249)"
EWMA,,,"(1.021, 0.307)","(1.041, 0.298)","(-0.604, 0.546)","(0.087, 0.931)"
GARCH_NORM,,,,"(0.894, 0.371)","(-0.973, 0.331)","(-0.414, 0.679)"
GARCH_STUD,,,,,"(-0.99, 0.322)","(-0.519, 0.604)"
GARCH_MIDAS_NORM,,,,,,"(1.34, 0.18)"
GARCH_MIDAS_STUD,,,,,,


In [14]:
print('Two-Daily MSE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/two_daily_mse_v3.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Two-Daily MSE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(1.677, 0.093)","(1.581, 0.114)","(1.582, 0.114)","(1.047, 0.295)","(1.315, 0.189)"
EWMA,,,"(0.715, 0.474)","(0.913, 0.361)","(-0.727, 0.468)","(-0.106, 0.916)"
GARCH_NORM,,,,"(1.418, 0.156)","(-0.926, 0.354)","(-0.4, 0.689)"
GARCH_STUD,,,,,"(-0.983, 0.326)","(-0.559, 0.576)"
GARCH_MIDAS_NORM,,,,,,"(1.358, 0.175)"
GARCH_MIDAS_STUD,,,,,,


In [15]:
print('Weekly MSE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/weekly_mse_v3.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Weekly MSE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(1.838, 0.066)","(1.689, 0.091)","(1.71, 0.087)","(1.495, 0.135)","(1.613, 0.107)"
EWMA,,,"(-0.311, 0.756)","(0.612, 0.541)","(-0.936, 0.349)","(0.082, 0.934)"
GARCH_NORM,,,,"(1.768, 0.077)","(-1.077, 0.282)","(0.414, 0.679)"
GARCH_STUD,,,,,"(-1.266, 0.206)","(-0.468, 0.64)"
GARCH_MIDAS_NORM,,,,,,"(1.569, 0.117)"
GARCH_MIDAS_STUD,,,,,,


In [16]:
print('Two-weekly MSE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/two_weekly_mse_v3.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Two-weekly MSE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(1.982, 0.047)","(1.819, 0.069)","(1.845, 0.065)","(1.726, 0.084)","(1.788, 0.074)"
EWMA,,,"(-0.741, 0.459)","(0.377, 0.706)","(-0.732, 0.464)","(0.392, 0.695)"
GARCH_NORM,,,,"(1.998, 0.046)","(-0.712, 0.476)","(1.333, 0.182)"
GARCH_STUD,,,,,"(-1.446, 0.148)","(0.375, 0.708)"
GARCH_MIDAS_NORM,,,,,,"(1.856, 0.063)"
GARCH_MIDAS_STUD,,,,,,


In [17]:
print('Monthly MSE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/monthly_mse_v3.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Monthly MSE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(1.985, 0.047)","(1.833, 0.067)","(1.855, 0.064)","(1.809, 0.071)","(1.834, 0.067)"
EWMA,,,"(-1.027, 0.304)","(0.118, 0.906)","(-0.77, 0.442)","(0.425, 0.671)"
GARCH_NORM,,,,"(1.962, 0.05)","(0.12, 0.905)","(1.611, 0.107)"
GARCH_STUD,,,,,"(-1.718, 0.086)","(0.916, 0.36)"
GARCH_MIDAS_NORM,,,,,,"(1.911, 0.056)"
GARCH_MIDAS_STUD,,,,,,


In [18]:
print('Daily QLIKE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/daily_qlike_v3.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Daily QLIKE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(1.756, 0.079)","(2.5, 0.012)","(2.251, 0.024)","(2.521, 0.012)","(2.361, 0.018)"
EWMA,,,"(1.994, 0.046)","(1.714, 0.087)","(2.108, 0.035)","(1.916, 0.055)"
GARCH_NORM,,,,"(-1.923, 0.055)","(0.28, 0.779)","(-0.302, 0.763)"
GARCH_STUD,,,,,"(1.184, 0.237)","(0.785, 0.432)"
GARCH_MIDAS_NORM,,,,,,"(-1.711, 0.087)"
GARCH_MIDAS_STUD,,,,,,


In [19]:
print('Two-Daily QLIKE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/two_daily_qlike_v3.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Two-Daily QLIKE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(1.261, 0.207)","(2.42, 0.016)","(1.934, 0.053)","(2.417, 0.016)","(2.112, 0.035)"
EWMA,,,"(3.213, 0.001)","(2.3, 0.021)","(3.445, 0.001)","(2.768, 0.006)"
GARCH_NORM,,,,"(-4.21, 0.0)","(0.405, 0.686)","(-0.699, 0.484)"
GARCH_STUD,,,,,"(2.16, 0.031)","(1.171, 0.242)"
GARCH_MIDAS_NORM,,,,,,"(-4.102, 0.0)"
GARCH_MIDAS_STUD,,,,,,


In [20]:
print('Weekly QLIKE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/weekly_qlike_v3.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Weekly QLIKE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(0.636, 0.525)","(1.884, 0.06)","(1.345, 0.178)","(1.927, 0.054)","(1.58, 0.114)"
EWMA,,,"(3.468, 0.001)","(2.414, 0.016)","(4.68, 0.0)","(3.49, 0.0)"
GARCH_NORM,,,,"(-4.729, 0.0)","(1.017, 0.309)","(-0.261, 0.794)"
GARCH_STUD,,,,,"(3.123, 0.002)","(1.733, 0.083)"
GARCH_MIDAS_NORM,,,,,,"(-6.361, 0.0)"
GARCH_MIDAS_STUD,,,,,,


In [21]:
print('Two-Weekly QLIKE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/two_weekly_qlike_v3.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Two-Weekly QLIKE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(0.46, 0.645)","(1.644, 0.1)","(1.233, 0.218)","(1.746, 0.081)","(1.475, 0.14)"
EWMA,,,"(2.363, 0.018)","(1.719, 0.086)","(3.039, 0.002)","(2.372, 0.018)"
GARCH_NORM,,,,"(-3.88, 0.0)","(1.258, 0.209)","(0.229, 0.819)"
GARCH_STUD,,,,,"(2.947, 0.003)","(1.969, 0.049)"
GARCH_MIDAS_NORM,,,,,,"(-3.732, 0.0)"
GARCH_MIDAS_STUD,,,,,,


In [22]:
print('Monthly QLIKE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/Monthly_qlike_v3.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Monthly QLIKE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(0.135, 0.893)","(1.456, 0.145)","(1.039, 0.299)","(1.511, 0.131)","(1.241, 0.215)"
EWMA,,,"(2.19, 0.029)","(1.603, 0.109)","(2.775, 0.006)","(2.14, 0.032)"
GARCH_NORM,,,,"(-4.276, 0.0)","(0.998, 0.318)","(-0.076, 0.939)"
GARCH_STUD,,,,,"(3.037, 0.002)","(1.84, 0.066)"
GARCH_MIDAS_NORM,,,,,,"(-3.806, 0.0)"
GARCH_MIDAS_STUD,,,,,,
